# KIX 4F Simulation (yokhr)

## Imports and Initializations

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib.pyplot as plt

from copy import deepcopy

from src.simfunc.simulation import Simulation  # class for simulation
from src.utils.simparam import SimParam  # class for simulation parameters
from src.utils.utils import day_graph, minutes_to_hms

%matplotlib inline
%config InlineBackend.figure_format='retina'

## Based on Terminal-1 6kPAX Demand

### Preparation

In [ ]:
simparam6k = SimParam()
path_to_6k_sched = (
    r"/home/yokhr/projects/KAPpaxsim/data/processed/Schedule "
    r"(30th terminal peak, 6000 pax)_PROCESSED_yokhr.xlsx"
    # Directly remove T2 airlines: APJ, JJA, CQH
)
simparam6k.schedule_from_path(path_to_6k_sched)
simparam6k.schedule_cleanup().schedule_filter().assign_check_in()
simparam6k.show_up_from_file().assign_flight_show_up_category_default().assign_show_up()


simparam6k.plot_name = "T1 6kPAX"
simparam6k.plot_std()
simparam6k.plot_counters()

### With CU for airlines using many counters during peak hours

In [ ]:
loc_start = 80  # 0 should be set for the start of the day
loc_end = 240  # 288 should be set for the end of the day
CU_AL_Nms = [0, 5, 10, 12, 15, 20, 30, 40, 45]

x = []
y = []

for CU_AL_Nm in CU_AL_Nms:
    simparam = simparam6k
    simparam_tradi = deepcopy(simparam)

    simparam.df_Counters["total"].plot()
    simparam.df_Counters["total"].loc[loc_start:loc_end,].plot()

    all_airlines = simparam.df_Counters.iloc[:, :-1].sum(axis=0).index.to_list()
    airlines_CU_6k = (
        simparam.df_Counters.loc[loc_start:loc_end,]
        .sum(axis=0)
        .sort_values(ascending=False)
        .index.to_list()[2 : CU_AL_Nm + 2]  # to skip "NEW"
    )
    airlines_tradi = [x for x in all_airlines if x not in airlines_CU_6k]

    mask_airlines_CU_6k = simparam.schedule["Airline Code"].isin(airlines_CU_6k)

    simparam_tradi.schedule = simparam_tradi.schedule[
        mask_airlines_CU_6k == False
    ].copy()
    simparam_tradi.assign_check_in()

    # Plot graph
    print(f"CU airlines are: {len(airlines_CU_6k)} {airlines_CU_6k}")
    print(f"Tradi airlines are: {len(airlines_tradi)} {sorted(airlines_tradi)}")
    print(f"Max used tradi counters: {simparam_tradi.df_Counters['total'].max()}")
    simparam_tradi.plot_name = f"{len(airlines_tradi)} Tradi airlines"
    simparam_tradi.plot_counters(compare_with=simparam)

    x = x + [CU_AL_Nm]
    y = y + [simparam_tradi.df_Counters["total"].max()]

plt.xlabel("AL with CU")
plt.ylabel("Max tradi Ctr")
plt.xticks(x)
plt.plot(x, y, marker="o", linestyle="dotted")

### With CU for selected airlines

In [ ]:
selected_ALs = [
    [  # Pattern 1: All Green for CUSBD & CUCkIn
        "AAR",
        "ACA",
        "CCA",
        "CPA",
        "CQH",
        "CRK",
        "CSC",
        "CSZ",
        "CXA",
        "EVA",
        "GCR",
        "JJA",
        "JJP",
        "JST",
        "SJX",
        "TAX",
        "XAX",
    ],
    [  # Pattern 2: Not Negative for CUSBD & CUCkIn
        "AAR",
        "ACA",
        "AMU",
        "ANA",
        "ANZ",
        "BAW",
        "CAL",
        "CBJ",
        "CCA",
        "CEB",
        "CES",
        "CPA",
        "CQH",
        "CRK",
        "CSC",
        "CSH",
        "CSN",
        "CSZ",
        "CXA",
        "DKH",
        "DLH",
        "EVA",
        "FIN",
        "GCR",
        "GIA",
        "HKE",
        "JJA",
        "JJP",
        "JNA",
        "JSA",
        "JST",
        "OKA",
        "SJX",
        "SWR",
        "TAX",
        "THA",
        "TLM",
        "UAE",
        "XAX",
    ],
    [  # Pattern 3: Agree on CUSBD
        "AAR",
        "ABL",
        "ACA",
        "AMU",
        "ANA",
        "ANZ",
        "CAL",
        "CEB",
        "CES",
        "CPA",
        "CQH",
        "CSC",
        "CSH",
        "CXA",
        "DKH",
        "EVA",
        "FIN",
        "GCR",
        "HKE",
        "JAL",
        "JJA",
        "JJP",
        "JNA",
        "JSA",
        "JST",
        "KAL",
        "MAS",
        "OKA",
        "SIA",
        "SJX",
        "TAX",
        "TWB",
        "UAE",
        "XAX",
    ],
    [  # Pattern 4: Not disagree on CUSBD
        "AAR",
        "ABL",
        "ACA",
        "AFR",
        "AMU",
        "ANA",
        "ANZ",
        "CAL",
        "CCA",
        "CDC",
        "CEB",
        "CES",
        "CPA",
        "CQH",
        "CRK",
        "CSC",
        "CSH",
        "CSZ",
        "CXA",
        "DKH",
        "EVA",
        "FIN",
        "GCR",
        "GIA",
        "HKE",
        "JAL",
        "JJA",
        "JJP",
        "JNA",
        "JSA",
        "JST",
        "KAL",
        "KLM",
        "MAS",
        "OKA",
        "SIA",
        "SJX",
        "TAX",
        "TWB",
        "UAE",
        "XAX",
    ],
]

In [ ]:
for airlines_CU_6k in selected_ALs:
    simparam = simparam6k
    simparam_tradi = deepcopy(simparam)

    all_airlines = simparam.df_Counters.iloc[:, :-1].sum(axis=0).index.to_list()
    airlines_tradi = [x for x in all_airlines if x not in airlines_CU_6k]
    mask_airlines_CU = simparam.schedule["Airline Code"].isin(airlines_CU_6k)
    simparam_tradi.schedule = simparam_tradi.schedule[mask_airlines_CU == False].copy()
    simparam_tradi.assign_check_in()

    # Plot graph
    print(f"CU airlines are: {len(airlines_CU_6k)} {airlines_CU_6k}")
    print(f"Tradi airlines are: {len(airlines_tradi)} {sorted(airlines_tradi)}")
    print(f"Max used tradi counters: {simparam_tradi.df_Counters['total'].max()}")

    simparam_tradi.plot_name = f"{len(airlines_tradi)} Tradi airlines"

    simparam_tradi.plot_counters(compare_with=simparam)

### Processing Time Simulation

In [ ]:
# Pattern 3
airline_CU_6k = selected_ALs[2]

simparam6k.df_Pax["pax_type"] = simparam6k.df_Pax["Airline"].apply(
    lambda x: "CUSBD" if x in airlines_CU_6k else "tradi"
)
simparam6k.schedule["pax_type"] = simparam6k.schedule["Airline Code"].apply(
    lambda x: "CUSBD" if x in airlines_CU_6k else "tradi"
)
simparam6k.plot_std(by_col=True, col_name="pax_type")

In [ ]:
# Resource Number, Processing Time, Opening Hour...
RNm_kiosk = 92  # Resource number of Kiosk
RNm_CUSBDs = [32, 40, 41, 42, 50, 51, 52]  # Resource number of CUSBD
Ptm_kiosk = 70  # Process time of Kiosk
Ptm_CUSBD = 60  # Process time of CUSBD
Ptm_CkIn = 80  # Process time of CheckIn
HrSTD_CUSBDs = [2.5, 3.0, 3.5]  # Hours to STD of CUSBD
HrSTD_CkIn = 2.5  # Hours to STD of CheckIn

for RNm_CUSBD in RNm_CUSBDs:
    for HrSTD_CUSBD in HrSTD_CUSBDs:
        simparam6k.dct_resource = {
            "kiosk": RNm_kiosk,
            "CUSBD": RNm_CUSBD,
        }
        simparam6k.dct_processes = {
            "kiosk": Ptm_kiosk,
            "CUSBD": Ptm_CUSBD,
            "checkin": Ptm_CkIn,
        }
        simparam6k.dct_process_sequence = {
            "tradi": [
                ["kiosk", "wait_opening", "checkin"],
                {"hour_to_std": HrSTD_CkIn},
            ],
            "CUSBD": [
                ["kiosk", "wait_opening", "CUSBD"],
                {"hour_to_std": HrSTD_CUSBD},
            ],
        }

        print(f"The number of CUSBD: {RNm_CUSBD}")
        print(f"Opening hrs before STD [CUSBD]: {HrSTD_CUSBD} hr")
        print(f"Opening hrs before STD [CkIn]: {HrSTD_CkIn} hr")

        simulation6k = Simulation(simparam6k)
        simulation6k.generate_checkin().generate_pax().run()
        simulation6k.format_df_result()
        simulation6k.plot_result()

        wtCUSBD = round(simulation6k.dct_hist_wait_time["CUSBD"].quantile(0.9), 2)
        print(f"Top 90% waiting time [CUSBD]: {wtCUSBD} min")
        wtCkIn = round(simulation6k.dct_hist_wait_time["checkin"].quantile(0.9), 2)
        print(f"Top 90% waiting time [CkIn]: {wtCkIn} min")